In [27]:
import numpy as np 
import pandas as pd 
data = pd.read_csv('edited_data.csv')
X = data['Objectives']+'. '+ data['Description']
X = X.str.replace('<ul>', '').str.replace('<li>','').str.replace('</li>','').\
str.replace('</ul>','').str.replace('1.','').str.replace('2.','').str.replace('3.','').str.replace('4.','').str.replace('5.','').\
str.replace('(','').str.replace(')','').str.replace('\n','')
X = X.fillna('')

In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1
x = pad_sequences(sequences, padding='post', maxlen=80)
print(x[0, :])

[4312    3 3020 1764 3021   41  501  789   29 4313    8  533   69  747
 2020    8  501   69  747    7  428   11    8 3020 1764 3021    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

y2 = data['Project Tag']
mlb = MultiLabelBinarizer()
y2 = y2.fillna("N/A")
y2 = y2.str.split(', ')

for i in range(0, len(y2)):
    for j in range(0, len(y2[i])):
        y2[i][j] = y2[i][j].replace("Community Event ", "Community Event")\
        .replace("Conference/Panel Discussion ", "Conference/Panel Discussion")\
        .replace("Educational Material ", "Educational Material")\
        .replace("Social Media ", "Social Media").replace("Survey ", "Survey")\
        .replace("Teaching Activity ", "Teaching Activity")
    
mlb_y2 = mlb.fit_transform(y2)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, mlb_y2, test_size=0.3, random_state=52)

#print(len(mlb.classes_))

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D, LSTM
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

filter_length = 300

model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                           output_dim= 50, input_length=90))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(len(mlb.classes_)))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = model.fit(X_train, y_train,
                    #class_weight=class_weight,
                    epochs=20,
                    batch_size=300,
                    validation_split=0.1,
                    callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 90, 50)            379650    
_________________________________________________________________
dropout_12 (Dropout)         (None, 90, 50)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 88, 300)           45300     
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 17)                5117      
_________________________________________________________________
activation_6 (Activation)    (None, 17)                0         
Total params: 430,067
Trainable params: 430,067
Non-trainable params: 0
_________________________________________________________________
Trai

In [33]:
from keras import models
cnn_model = keras.models.load_model('model-conv1d.h5')
metrics = cnn_model.evaluate(X_test, y_test)
print("{}: {}".format(model.metrics_names[0], metrics[0]))
print("{}: {}".format(model.metrics_names[1], metrics[1]))

loss, accuracy = cnn_model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

646/646 [==============================] - 1s 810us/step
loss: 0.2147432404221396
categorical_accuracy: 0.35294117651672186
Training Accuracy: 0.3550
